In [1]:
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
from nltk.corpus import stopwords

import pandas as pd
import re
import numpy as np
import spacy
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
#import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhouya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
D:\python\Anaconda\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [2]:
from time import time
print("Loading dataset...")
t0 = time()

a=0 
b=0
c=0
data=[]
date_year=[]
data_ab=[]
f=open("savedrecs.txt","r",encoding='utf-8')
f.readline()
for line in f:
    #print (line.startswith('AB'))K1TI
    if line.startswith('TI') == True :
        if len(line.lstrip('TI').strip())>0:
            A=line.lstrip('TI').strip()
            a=1
            
    if line.startswith('PY') == True :
        if len(line.lstrip('PY').strip())>0:
            B=line.lstrip('PY').strip()
            b=2
            
    if line.startswith('AB') == True :
        if len(line.lstrip('AB').strip())>0:
            C=line.lstrip('AB').strip() 
            c=3

    if a==1 and b==2 and c==3:
        data.append(A) 
        date_year.append(B) 
        data_ab.append(C)     
        a=0
        b=0
        c=0
        
print("done in %0.3fs." % (time() - t0))


Loading dataset...
done in 0.082s.


In [3]:
print(len(data_ab))

1121


In [4]:
print(len( date_year))

1121


In [5]:
print(len(data))

1121


In [6]:
from pandas.core.frame import DataFrame
#列表b
c={"title" : data,
   "year" : date_year,
  "abstract" : data_ab}#将列表a，b转换成字典
DATA=DataFrame(c)#将字典转换成为数据框
print(DATA)


                                                  title  year  \
0     Semantic Representation of Patient Triage Data...  2021   
1     A systematic review of contributing factors of...  2021   
2                  Analysis of the new scopus CiteScore  2021   
3     Plants used against obesity in Turkish folk me...  2021   
4     An evidence-based synthesis of instructional r...  2021   
...                                                 ...   ...   
1116  Libraries Australia: creating a new national r...  2005   
1117  Using concept maps in digital libraries as a c...  2005   
1118  A hybrid declarative/procedural metadata mappi...  2005   
1119  The effect of simmering on the chemical and me...  2005   
1120  The challenge of digitization: Libraries are f...  2005   

                                               abstract  
0     Emergency Department (ED) overcrowding is a ma...  
1     Objective: Physicians often describe the elect...  
2     This study analyzes the new version of 

In [7]:
documents=list(DATA['title'])

In [8]:
time=DATA['year']
DATA['year']=time.str.slice(0,4) 

In [9]:
print(DATA['year'])

0       2021
1       2021
2       2021
3       2021
4       2021
        ... 
1116    2005
1117    2005
1118    2005
1119    2005
1120    2005
Name: year, Length: 1121, dtype: object


In [10]:
time_stamps=np.arange(int(DATA.year.min(axis=0)),int(DATA.year.max(axis=0))+1,1)
time_stamps=list(time_stamps)


# find out the time slice
gp=DATA.groupby(by=['year'])
total_yearly_list=list(gp.size())

In [11]:
print(time_stamps)
print(total_yearly_list)

[2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
[36, 34, 39, 43, 54, 43, 65, 88, 59, 71, 90, 82, 87, 87, 122, 99, 22]


In [12]:
stoplist=stopwords

stoplist = set('for a an of the with on and to in at from as by about is was where that = or were this their are we be through'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

In [13]:
import os
from collections import  defaultdict
# drop the words only appers once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

dictionary = corpora.Dictionary(texts)   # generate the dictionary
dictionary.compactify() #

dictionary.save(os.path.join('dictionary.dict')) # store the dictionary, for future reference
print(dictionary)


Dictionary(1372 unique tokens: ['collected', 'data', 'departments.', 'emergency', 'patient']...)


In [14]:
import os
#Save vocabulary
vocFile = open(os.path.join( 'vocabulary.dat'),'w',encoding='utf-8')
for word in dictionary.values():
    print(word)
    vocFile.write(word+'\n')
vocFile.close()
print(vocFile)


collected
data
departments.
emergency
patient
representation
semantic
triage
electronic
factors
health
impacts
physician
review
solutions
systematic
well-being
analysis
new
medicine:
obesity
plants
used
context
covid-19
evidence-based
instructional
procedures
rapid
reading
synthesis
using
academic
behavior
changing
libraries
library
professionals
role
documentation
implementation
nursing
safety
toward
departments
prevalence
us
visits
adult
availability
children's
diverse
e-book
need
platforms
young
aspects
narrative
stories:
within
cattle
course
evaluation
monitoring
national
novel
programme
training
alignment
breast
cancer
lung
media
news
regarding
scientific
study
there
comparison
department
disorder:
not
opioid
patients
seeking
substance
treatment
use
versus
ambulance
between
departments:
handover
healthcare
qualitative
among
anaphylaxis
trends
five
longitudinal
medline
query
retrieval
search
alzheimer's
cognitive
disease
lessons
studies
acute
adults'
alleviating
controlled
effect
m

In [15]:
#Prevent storing the words of each document in the RAM
class MyCorpus(object):
     def __iter__(self):
         for document in documents:
             # assume there's one document per line, tokens separated by whitespace
             yield dictionary.doc2bow(document.lower().split())


corpus_memory_friendly = MyCorpus()

multFile = open(os.path.join( 'foo-mult.dat'),'w')

for vector in corpus_memory_friendly: # load one vector into memory at a time
    multFile.write(str(len(vector)) + ' ')
    for (wordID, weigth) in vector:
        multFile.write(str(wordID) + ':' + str(weigth) + ' ')

    multFile.write('\n')
    
multFile.close()

print(multFile)



<_io.TextIOWrapper name='foo-mult.dat' mode='w' encoding='cp936'>


In [16]:
print(type(vocFile))

<class '_io.TextIOWrapper'>


In [17]:
time_slice=total_yearly_list
print(time_slice)

[36, 34, 39, 43, 54, 43, 65, 88, 59, 71, 90, 82, 87, 87, 122, 99, 22]


In [18]:
print(type(corpus_memory_friendly))

<class '__main__.MyCorpus'>


In [19]:
print(dictionary)

Dictionary(1372 unique tokens: ['collected', 'data', 'departments.', 'emergency', 'patient']...)


In [ ]:
from gensim.models import LdaSeqModel
#use LdaSeqModel to generate DTM results
#ldaseq = LdaSeqModel(corpus=corpus_memory_friendly, id2word=dictionary, time_slice=time_slice, num_topics=5)
ldaseq = LdaSeqModel(corpus=corpus_memory_friendly, id2word=dictionary, time_slice=time_slice, num_topics=6)
# for given time, the distriibution of each topic 
ldaseq.print_topics(time=1)
# for given topic the word distribution over time


D:\python\Anaconda\lib\site-packages\gensim\models\ldaseqmodel.py:1474: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))
D:\python\Anaconda\lib\site-packages\gensim\models\ldaseqmodel.py:293: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)
D:\python\Anaconda\lib\site-packages\gensim\models\ldaseqmodel.py:1474: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))
D:\python\Anaconda\lib\site-packages\gensim\models\ldaseqmodel.py:1474: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))
D:\python\Anaconda\lib\site-packages\gensim\models\ldaseqmodel.py:1474: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))
D:\python\Anaconda\lib\

In [ ]:
DTM_topic_0=ldaseq.print_topic_times(topic=0, top_terms=10)
DTM_topic_1=ldaseq.print_topic_times(topic=1, top_terms=10)
DTM_topic_2=ldaseq.print_topic_times(topic=2, top_terms=10)
DTM_topic_3=ldaseq.print_topic_times(topic=3, top_terms=10)
DTM_topic_4=ldaseq.print_topic_times(topic=4, top_terms=10)
DTM_topic_5=ldaseq.print_topic_times(topic=5, top_terms=10)

In [ ]:
def topic_time(DTM_topic,time_stamps):  
    for i in range(len(time_slice)-1):
        if i==0:
            temp_a1=pd.DataFrame(DTM_topic[i])
            temp_a2=pd.DataFrame(DTM_topic[i+1])
            temp_a1.columns = ['words', time_stamps[i]]
            temp_a2.columns = ['words', time_stamps[i+1]]
            temp_a1=pd.merge(temp_a1,temp_a2)
        else:
            temp_a2=pd.DataFrame(DTM_topic[i+1])
            temp_a2.columns = ['words', time_stamps[i+1]]
            temp_a1=pd.merge(temp_a1,temp_a2)
    topic_words_time=temp_a1
    return topic_words_time
   
topic1_words_time=topic_time(DTM_topic_0,time_stamps)
topic2_words_time=topic_time(DTM_topic_1,time_stamps)
topic3_words_time=topic_time(DTM_topic_2,time_stamps)
topic4_words_time=topic_time(DTM_topic_3,time_stamps)
topic5_words_time=topic_time(DTM_topic_4,time_stamps)
topic6_words_time=topic_time(DTM_topic_5,time_stamps)

In [ ]:
print(topic1_words_time)

In [ ]:
#plot the dynamic movement of topic 1
topic1_words=list(topic1_words_time['words'])
plt.figure()
for i in range(0,5):
    plt.plot(time_stamps, topic1_words_time.iloc[i,1:],marker=".",label=topic1_words[i])
#plt.xlim((-1, 2))
#plt.ylim((0, 0.02))
plt.legend(loc='best')
plt.title('Topic 1')
plt.savefig('Topic4-1.png',transparent=True)
plt.rcParams.update({'font.size': 15})     #设置图例字体大小
plt.show()

In [ ]:
#plot the dynamic movement of topic2
topic=topic2_words_time
topic_words=list(topic['words'])
plt.figure()
for i in range(0,5):
    plt.plot(time_stamps, topic.iloc[i,1:],marker=".",label=topic_words[i])
#plt.xlim((-1, 2))
#plt.ylim((0, 0.02))
plt.legend(loc='best')
plt.title('Topic 2')
plt.rcParams.update({'font.size': 15})     #设置图例字体大小
plt.savefig('Topic4-2.png',transparent=True)
plt.show()

In [ ]:

#plot the dynamic movement of topic3
topic=topic3_words_time
topic_words=list(topic['words'])
plt.figure()
for i in range(0,5):
    plt.plot(time_stamps, topic.iloc[i,1:],marker=".",label=topic_words[i])
#plt.xlim((-1, 2))
#plt.ylim((0, 0.02))
plt.legend(loc='best')
plt.rcParams.update({'font.size': 15})     #设置图例字体大小
plt.title('Topic 3')
plt.savefig('Topic4-3.png',transparent=True)
plt.show()

In [ ]:
#plot the dynamic movement of topic4
topic=topic4_words_time
topic_words=list(topic['words'])
plt.figure()
for i in range(0,5):
    plt.plot(time_stamps, topic.iloc[i,1:],marker=".",label=topic_words[i])
#plt.xlim((-1, 2))
#plt.ylim((0, 0.02))
plt.legend(loc='best')
plt.title('Topic 4')
plt.rcParams.update({'font.size': 15})     #设置图例字体大小
plt.savefig('Topic4-4.png',transparent=True)
plt.show()

In [ ]:
#plot the dynamic movement of topic4
topic=topic5_words_time
topic_words=list(topic['words'])
plt.figure()
for i in range(0,5):
    plt.plot(time_stamps, topic.iloc[i,1:],marker=".",label=topic_words[i])
#plt.xlim((-1, 2))
#plt.ylim((0, 0.02))
plt.legend(loc='best')
plt.title('Topic 5')
plt.rcParams.update({'font.size': 15})     #设置图例字体大小
plt.savefig('Topic4-5.png',transparent=True)
plt.show()

In [ ]:
#plot the dynamic movement of topic4
topic=topic6_words_time
topic_words=list(topic['words'])
plt.figure()
for i in range(0,5):
    plt.plot(time_stamps, topic.iloc[i,1:],marker=".",label=topic_words[i])
#plt.xlim((-1, 2))
#plt.ylim((0, 0.02))
plt.legend(loc='best')
plt.title('Topic 6')
plt.rcParams.update({'font.size': 15})     #设置图例字体大小
plt.savefig('Topic4-6.png',transparent=True)
plt.show()